In [ ]:
import sys
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt

sys.path.append("..")
matplotlib.rc_file('../rc_file')
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

from Pico import PicoModel